### Cross-Validation and the Bootstrap

In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split

In [19]:
from functools import partial
from sklearn.model_selection import \
(cross_validate,
KFold,
ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

#### the Validation Set Approach
##### -split the data into training, validation set 
###### **Do not reuse the data used in traing or validation in test session**

- data introduction

mpg: miles per gallon

cylinders: Number of cylinders between 4 and 8

displacement: Engine displacement (cu. inches)

horsepower: Engine horsepower

weight: Vehicle weight (lbs.)

acceleration: Time to accelerate from 0 to 60 mph (sec.)

year: Model year (modulo 100)

origin: Origin of car (1. American, 2. European, 3. Japanese)

name: Vehicle name

A data frame with 392 observations on the following 9 variables.


In [4]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0) 
#Random_state = int
#Controls the shuffling applied to the data before applying the split. Pass an int for reproducible output across multiple function calls.
#Default = None

In [5]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

In [7]:
sum = summarize(results)
sum

,coef,std err,t,P>|t|
intercept,39.9055,1.009,39.537,0.0
horsepower,-0.1563,0.009,-17.333,0.0


In [8]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)

#Mean Squered Error of model (true - predicted value)
np.mean((y_valid - valid_pred)**2)

23.616617069669882

In [11]:
def evalMSE(terms, response, train, test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)

In [13]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower',degree)], 'mpg', Auto_train, Auto_valid) 
MSE
#poly function for creating polynomial function
#poly(variable, degree) -> sqaring the variable with the degree amounts
#in this case, 'horsepower' variable will be sqaured until 3 degress

array([23.61661707, 18.76303135, 18.79694163])

In [16]:
#from here we'll apply another training/validation split
Auto_train, Auto_valid = train_test_split(Auto, test_size = 106, random_state = 3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
MSE

array([23.73802814, 17.59231544, 17.53555995])

it seems like random shuffling the data set improves the prediction <br>
also, poly nomial term does too. But infinitely increasing the degree doesn't improve the performance.

### Cross-Validation

In [21]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns = ['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model, X, Y, cv= Auto.shape[0]) #Auto.shape[0] = LOOCV(same as data set length)
cv_err = np.mean(cv_results['test_score'])
cv_err

24.231513517929226

In [31]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H,np.arange(d+1)) #retrieve the powered values of H variable with amounts of 'd'
    M_CV = cross_validate(M,X, Y, cv = Auto.shape[0]) #LOOCV
    cv_error[i] = np.mean(M_CV['test_score']) #not a MSE, here, just averaging the test score
cv_error
#test_score in each poly nomial degree will have n amounts of test_score value(cause it's LOOCV)

array([24.23151352, 19.24821312, 19.33498406, 19.42443032, 19.03322078])

In [32]:
cv_error = np.zeros(5)
cv = KFold(n_splits = 10, shuffle = True, random_state = 0)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, Y, cv = cv) #10-fold cross-validation
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.478484  , 19.1372021 ])

In [33]:
validation = ShuffleSplit(n_splits =1, test_size = 196, random_state = 0) #shuffle data only once
results = cross_validate(hp_model, Auto.drop(['mpg'],axis = 1), Auto['mpg'], cv = validation)
results['test_score']

array([23.61661707])

In [35]:
validation = ShuffleSplit(n_splits =10, test_size = 196, random_state= 0)
results = cross_validate(hp_model, Auto.drop(['mpg'],axis = 1), Auto['mpg'], cv = validation)
results['test_score'].mean(), results['test_score'].std()

(23.802232661034164, 1.4218450941091783)

### Bootstrap

resampling technique that helps in estimating the uncertainty of a statistical model.

In [38]:
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx],rowvar=False)
    return ((cov_[1,1]-cov_[0,1]) / (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

cov_[1,1] = variance of Y <br>
cov_[0,1] or cov_[1,0] = covariance between X, Y <br>
cov_[0,0] = variance of X

In [39]:
alpha_func(Portfolio, range(100))

0.57583207459283

In [41]:
rng = np.random.default_rng(0)
alpha_func(Portfolio, rng.choice(100, 100, replace = True)) 
#Select 100 elements from [0,99]
#Bootstrap is copied data with replacement

0.6074452469619002

In [42]:
def boot_SE(func, D, n = None, B = 100, seed = 0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0,0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index, n ,replace = True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
        
        return np.sqrt(second_ / B - (first_ / B)**2)

In [43]:
alpha_SE = boot_SE(alpha_func, Portfolio, B = 1000 , seed = 0)
alpha_SE

0.019199498387420105

---

In [44]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params